In [ ]:
# 한국어 형태소 분석기 kiwi 설치 
#!pip install kiwipiepy

In [1]:
import requests

res = requests.get('https://github.com/e9t/nsmc/raw/master/ratings_train.txt')

with open('nsmc_train.csv', 'wb') as f:
    f.write(res.content)

In [22]:
import pandas as pd 
nsmc = pd.read_csv('nsmc_train.csv', sep='\t')

In [23]:
nsmc.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [24]:
from kiwipiepy import Kiwi
kiwi = Kiwi()
kiwi.prepare()

0

In [25]:
text = nsmc.document[0]

In [26]:
result = kiwi.analyze(text)

In [27]:
result[0][0]

[('아', 'IC', 0, 1),
 ('더빙', 'NNG', 2, 2),
 ('..', 'SF', 4, 2),
 ('진짜', 'MAG', 7, 2),
 ('짜증', 'NNG', 10, 2),
 ('나', 'VV', 12, 1),
 ('네요', 'EF', 13, 2),
 ('목소리', 'NNG', 16, 3)]

In [28]:
for token, pos, start, end in result[0][0]:
    if pos[0] in 'NV':
        print(f'{token}/{pos}')

더빙/NNG
짜증/NNG
나/VV
목소리/NNG


In [29]:
def extract_keywords(text):
    result = kiwi.analyze(text)
    for token, pos, start, end in result[0][0]:
        if pos[0] in 'NV':
            yield f'{token}/{pos}'

In [30]:
list(extract_keywords(text))

['더빙/NNG', '짜증/NNG', '나/VV', '목소리/NNG']

In [31]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [32]:
cv = CountVectorizer(max_features=1000, tokenizer=extract_keywords)
dtm = cv.fit_transform(nsmc.loc[0:1999, 'document'])

In [33]:
trans = TfidfTransformer()
dtm2 = trans.fit_transform(dtm)

In [34]:
words = cv.get_feature_names()

In [35]:
import joblib
joblib.dump({'words':words, 'dtm':dtm, 'dtm2':dtm2}, 'nsmc.pkl')

['nsmc.pkl']